In [1]:
from gensim.models import KeyedVectors
from tqdm import tqdm
from scipy.stats import pearsonr
from scipy.stats import spearmanr
import re
import numpy as np
import pandas as pd
import multiprocessing

In [ ]:
spxEM = '/home/manni/embs/en_wiki_spx_mc100_epoch5_300_em.txt'
#spxEM = '/home/manni/embs/en_wiki_spx_mc100_epoch5_300_em_w5_2.txt'
spxEM_model = KeyedVectors.load_word2vec_format(spxEM, binary=False)
w2v = '/home/manni/embs/en_wiki_w2v_mc100_epoch5_300.txt'
#w2v = '/home/manni/embs/en_wiki_w2v_mc100_epoch5_300_w5_2.txt'
w2v_model = KeyedVectors.load_word2vec_format(w2v, binary=False)
spx = '/home/manni/embs/en_wiki_spx_mc100_epoch5_300.txt'
spx_model = KeyedVectors.load_word2vec_format(spx, binary=False)
spxPEM = '/home/manni/embs/en_wiki_spx_mc100_epoch5_300_pem.txt'
spxPEM_model = KeyedVectors.load_word2vec_format(spxPEM, binary=False)
#spxNEM = '/home/manni/embs/en_wiki_spx_mc100_epoch5_300_nem.txt'
#spxNEM_model = KeyedVectors.load_word2vec_format(spxNEM, binary=False)

In [10]:
wnet = '/home/manni/embs/en_wiki_wnet_epoch5_300_w10_explicit.txt_trimed'
wnet_model = KeyedVectors.load_word2vec_format(wnet, binary=False)

In [11]:
dconf = '/home/manni/wn3.0_sense_vectors.txt_trimed'
dconf_model = KeyedVectors.load_word2vec_format(dconf, binary=False)

In [ ]:
models = [w2v_model,spx_model,spxEM_model,spxPEM_model,spxNEM_model]
model_names = ['Word2vec','EWEM-MIX','EWEM','EWEM-3',"EWEM-NULL"]

In [ ]:
spxEM = '/home/manni/embs/en_wiki_spx_mc100_epoch5_300_em_w1.txt'
spxEM_model = KeyedVectors.load_word2vec_format(spxEM, binary=False)
w2v = '/home/manni/embs/en_wiki_w2v_mc100_epoch5_300_w1.txt'
w2v_model = KeyedVectors.load_word2vec_format(w2v, binary=False)

In [12]:
models = [wnet_model,dconf_model]
model_names = ['wnet','dconf']

In [13]:
print('cut#1' in dconf_model)
print('cut#2' in dconf_model)

True
True


# SCWS

In [ ]:
scws_file = '/home/manni/data/wordsim/SCWS/ratings.txt'

In [ ]:
data = list()
with open(scws_file) as fin:
    lines = fin.readlines()
    for l in lines:
        l = l.lower().split('\t')
        w1 = l[1]
        w2 = l[3]
        c1 = l[5]
        c2 = l[6]
        score = float(l[7])
        data.append([w1,w2,c1,c2,score])

In [ ]:
total = len(data)
missing = 0
model = dconf_model
scores = list()
truths = list()
for d in tqdm(data,position=0):
    score=avg_sim(d[0],d[1],model)
    if score:
        scores.append(score)
        truths.append(float(d[-1]))
    else:
        missing+=1
coverage = ((total-missing)/total)*100
print(round(coverage,2))
print(spearmanr(truths, scores))

In [ ]:
total = len(data)
missing = 0
model = spxPEM_model
scores = list()
truths = list()
for d in tqdm(data,position=0):
    score=avg_sim(d[0],d[1],model)
    if score:
        scores.append(score)
        truths.append(float(d[-1]))
    else:
        missing+=1
coverage = ((total-missing)/total)*100
print(round(coverage,2))
print(spearmanr(truths, scores))

In [ ]:
d = data[37]
d

In [ ]:
print(avg_sim(d[0],d[1],dconf_model))
print(avg_sim(d[0],d[1],spxPEM_model))
print(avg_sim(d[0],d[1],w2v_model))

In [ ]:
from nltk.corpus import stopwords
stop_words = set(stopwords.words('english'))

def clean_sent(sent):
    '''
    parameters:
        sent: str
    returns:
        tokens: list(str)     
    '''
    return [word for word in sent.split() if word not in stop_words]

def sent_words(sent):
    '''
    parameters:
        sent: str
    returns:
        tokens: list(str) 
    '''
    sent = re.sub(r'[^\w\s]', '', sent)
    left = sent.split('<b>')[0]
    right = sent.split('</b>')[-1]
    sentA = clean_sent(left)[-10:]
    sentB = clean_sent(right)[:10]
    return sentA+sentB

def reducer(wrd,lst,model):
    out = set()
    for word in lst:
        top = [tup[0] for tup in model.most_similar(word,topn=100)]
        out.update(set(top)&set(lst))
    out.add(wrd)
    if wrd+'#E' in model.vocab:
        out.add(wrd+'#E')
    return out

def avgsim(w1,w2,c1,c2,score,model,reduce=False,verbose=False):
    '''
    Computes average similarity score.

    parameters:
        w1 : str
        w2 : str
        c1 : str
        c2 : str
        model : gensim.keyedvectors
    returns:
        score : float
    '''
    if w1 not in model.vocab or w2 not in model.vocab:
        return None
    a = [w1] + sent_words(c1)
    _a = [word+'#E' for word in a] 
    a+=_a
    b = [w2] + sent_words(c2)
    _b = [word+'#E' for word in b] 
    b+=_b
    a = [word for word in a if word in model.vocab]
    b = [word for word in b if word in model.vocab]
    if reduce:
        a = reducer(w1,a,model)
        b = reducer(w2,b,model)    
    if verbose:
        print(a)
        print(b)
    div = len(a)*len(b)
    sims = 0
    for i in a:
        for j in b:
            sim = model.similarity(i,j)
            if verbose:
                print(i,j,sim)
            sims+= sim
    return (sims/div,score)

def _avgvec(lst,model):
    vec = np.zeros(model.vector_size)
    for word in lst:
        if word in model:
            vec = np.add(vec,model.get_vector(word))
    return vec  
    
def avgvec(w1,w2,c1,c2,score,model,reduce=False,verbose=False):
    if w1 not in model.vocab or w2 not in model.vocab:
        return None
    a = [w1] + sent_words(c1)
    _a = [word+'#E' for word in a] 
    a+=_a
    b = [w2] + sent_words(c2)
    _b = [word+'#E' for word in b] 
    b+=_b
    a = [word for word in a if word in model.vocab]
    b = [word for word in b if word in model.vocab]
    if reduce:
        a = reducer(w1,a,model)
        b = reducer(w2,b,model) 
    if verbose:
        print(a)
        print(b) 
    v1 = _avgvec(a,model)
    v2 = _avgvec(b,model)
    return (model.cosine_similarities(v1,[v2])[0],score)

In [ ]:
def get_corr(model,method='asim',reduce=False):
    sims = list()
    scores = list()
    for dat in tqdm(data,position=0):
        if method == 'asim':
            sim = avgsim(dat[0],dat[1],dat[2],dat[3],dat[4],model,reduce=reduce)
        if method == 'avg':
            sim = avgvec(dat[0],dat[1],dat[2],dat[3],dat[4],model,reduce=reduce)
        if sim:
            sims.append(sim[0])
            scores.append(sim[1])
    corr, _ = pearsonr(sims, scores)
    print('%.2f &' % (corr*100),end=' ')
    corr, _ = spearmanr(sims, scores)
    print('%.2f' % (corr*100))   

In [ ]:
get_corr(spxPEM_model)
get_corr(spxEM_model)
get_corr(spx_model)
get_corr(w2v_model)

In [ ]:
get_corr(spxPEM_model,method='avg')
get_corr(spxEM_model,method='avg')
get_corr(spx_model,method='avg')
get_corr(w2v_model,method='avg')

In [ ]:
from multiprocessing import Pool

def get_corr_parallel(model,method='asim',reduce=False):
    if __name__=="__main__":
        pool = Pool(processes=40)
        sims = list()
        scores = list()
        for dat in tqdm(data,position=0):
            if method == 'asim':
                sim = pool.apply_async(func=avgsim, args=(dat[0],dat[1],dat[2],dat[3],dat[4],model,reduce)) 
            if method == 'avg':
                sim = pool.apply_async(func=avgvec, args=(dat[0],dat[1],dat[2],dat[3],dat[4],model,reduce)) 
            sim = sim.get()
            if sim:
                sims.append(sim[0])
                scores.append(sim[1])
        pool.close()
        pool.join()
        corr, _ = pearsonr(sims, scores)
        print('Pearsons correlation: %.2f' % (corr*100))
        corr, _ = spearmanr(sims, scores)
        print('Spearmans correlation: %.2f' % (corr*100)) 

In [ ]:
get_corr_parallel(spxEM_model,reduce=True)
get_corr_parallel(spx_model,reduce=True)
get_corr_parallel(w2v_model,reduce=True)

In [ ]:
get_corr(spxEM_model,reduce=True)
get_corr(spx_model,reduce=True)
get_corr(w2v_model,reduce=True)

In [ ]:
get_corr(spxEM_model,method='avg',reduce=True)
get_corr(spx_model,method='avg',reduce=True)
get_corr(w2v_model,method='avg',reduce=True)

In [ ]:
print(w2v_model.similarity('brazil','nut'))
print(spx_model.similarity('brazil','nut'))
print(spxEM_model.similarity('brazil','nut'))
print(spxEM_model.similarity('brazil#E','nut#E'))

In [ ]:
print(spx_model.similarity('new','york'))
print(w2v_model.similarity('new','york'))
print(spxEM_model.similarity('new','york'))
print(spxEM_model.similarity('new#E','york#E'))

In [ ]:
avgsim(data[0][0],data[0][1],data[0][2],data[0][3],w2v_model,verbose=True)

In [ ]:
x = ['brazil', 'bahia', 'instance', 'never', 'gap', 'income', 'blacks', 'nonwhites', 'relatively', 'brazil#E', 'bahia#E', 'instance#E', 'never#E', 'gap#E', 'income#E', 'blacks#E', 'relatively#E']
y = ['nut', 'player', 'turns', 'change', 'string', 'tension', 'neck', 'bridge', 'pickups', 'features', 'found', 'nut#E', 'player#E', 'turns#E', 'change#E', 'string#E', 'tension#E', 'neck#E', 'bridge#E', 'pickups#E', 'features#E', 'found#E']

In [ ]:
x = ['brazil', 'population', 'black', 'politicians', 'city', 'salvador', 'bahia', 'instance', 'never', 'gap', 'income', 'blacks', 'nonwhites', 'relatively', 'small', 'compared', 'large', 'gap', 'whites', 'brazil#E', 'population#E', 'black#E', 'politicians#E', 'city#E', 'salvador#E', 'bahia#E', 'instance#E', 'never#E', 'gap#E', 'income#E', 'blacks#E', 'relatively#E', 'small#E', 'compared#E', 'large#E', 'gap#E', 'whites#E']
y = ['nut', 'machine', 'heads', 'worm', 'gears', 'player', 'turns', 'change', 'string', 'tension', 'neck', 'bridge', 'pickups', 'features', 'found', 'almost', 'every', 'guitar', 'photo', 'shows', 'nut#E', 'machine#E', 'heads#E', 'worm#E', 'gears#E', 'player#E', 'turns#E', 'change#E', 'string#E', 'tension#E', 'neck#E', 'bridge#E', 'pickups#E', 'features#E', 'found#E', 'almost#E', 'every#E', 'guitar#E', 'photo#E', 'shows#E']

In [ ]:
x = ['israel', 'case', 'incorporates', 'fiber', 'optics', 'list', 'pathogens', 'attached', 'silver', 'gold', 'nanowires', 'israel#E', 'case#E', 'fiber#E', 'optics#E', 'list#E', 'pathogens#E', 'attached#E', 'silver#E', 'gold#E']
y = ['israeli', 'policy', 'territories', 'song', 'banned', 'radio', 'israeli', 'folk', 'singer', 'miri', 'israeli#E', 'policy#E', 'territories#E', 'song#E', 'banned#E', 'radio#E', 'israeli#E', 'folk#E', 'singer#E', 'miri#E']

In [ ]:
x = ['israel', 'similar', 'widely', 'employed', 'immunological', 'technique', 'case', 'incorporates', 'fiber', 'optics', 'list', 'pathogens', 'attached', 'silver', 'gold', 'nanowires', 'netherlands', 'company', 'tno', 'designed', 'israel#E', 'similar#E', 'widely#E', 'employed#E', 'immunological#E', 'technique#E', 'case#E', 'fiber#E', 'optics#E', 'list#E', 'pathogens#E', 'attached#E', 'silver#E', 'gold#E', 'netherlands#E', 'company#E', 'tno#E', 'designed#E']
y = ['israeli', 'written', 'shalom', 'hanoch', 'protest', 'israeli', 'policy', 'territories', 'song', 'banned', 'radio', 'israeli', 'folk', 'singer', 'miri', 'aloni', 'sang', 'israeli', 'pop', 'song', 'israeli#E', 'written#E', 'shalom#E', 'hanoch#E', 'protest#E', 'israeli#E', 'policy#E', 'territories#E', 'song#E', 'banned#E', 'radio#E', 'israeli#E', 'folk#E', 'singer#E', 'miri#E', 'aloni#E', 'sang#E', 'israeli#E', 'pop#E', 'song#E']

In [ ]:
v1 = get_avgvec(x,w2v_model)
v2 = get_avgvec(y,w2v_model)

w2v_model.cosine_similarities(v1,[v2])

In [ ]:
v1 = get_avgvec(x,spx_model)
v2 = get_avgvec(y,spx_model)

spx_model.cosine_similarities(v1,[v2])

In [ ]:
v1 = get_avgvec(x,spxEM_model)
v2 = get_avgvec(y,spxEM_model)

spxEM_model.cosine_similarities(v1,[v2])

In [ ]:
avgsim(data[0][0],data[0][1],data[0][2],data[0][3],spx_model,verbose=True)

In [ ]:
avgsim(data[0][0],data[0][1],data[0][2],data[0][3],spxEM_model,verbose=True)

In [ ]:
avgsim(data[6][0],data[6][1],data[6][2],data[6][3],spxEM_model,verbose=True)

In [ ]:
for w in model.vocab:
    if '#' in w:
        print(w)
        break

In [ ]:
import wiki as w
import sys
sys.path.append("../../imports/")
import saver as sv

In [ ]:
sentences = sv.load("wiki_sentences_spx")

In [ ]:
for sent in sentences:
    print(sent[:105])
    break

In [ ]:
print(data[0][0])
print(sent_words(data[0][2]))
print(data[0][1])
print(sent_words(data[0][3]))

In [ ]:
spxEM_model.similarity('brazil','nut')

In [ ]:
spxEM_model.similarity('brazil#E','nut#E')

In [ ]:
w2v_model.similarity('brazil','nut')

In [ ]:
sent_words(c1)

# non-context similarities

In [4]:
ws353A = '/home/manni/data/wordsim/EN-WS353.out'
ws353R = '/home/manni/data/wordsim/EN-WSR353.out'
ws353S = '/home/manni/data/wordsim/EN-WSS353.out'
rw = '/home/manni/data/wordsim/rw.out'
sim999 = '/home/manni/data/wordsim/EN-SIM999.out'
turk = '/home/manni/data/wordsim/EN_TRUK.txt'
mturk = '/home/manni/data/wordsim/MTURK-771.out'
rg = '/home/manni/data/wordsim/EN-RG-65.txt'
men = '/home/manni/data/wordsim/EN-MEN-LEM.out'

In [ ]:
'''
dat = rw
with open(dat) as fin:
    lines = fin.readlines()
    with open(dat.split('.')[0]+'.out','w') as fout:
        for line in lines:
            line = line.split()
            try:
                float(line[2])
                fout.write("{} {} {} \n".format(line[0],line[1],line[2])) 
            except:
                continue 
'''

In [ ]:
'''
dat = men
with open(dat) as fin:
    lines = fin.readlines()
    with open(dat.split('.')[0]+'.out','w') as fout:
        for line in lines:
            line = line.strip().split()
            try:
                float(line[2])
                w1 = line[0].split('-')[0]
                w2 = line[1].split('-')[0]
                fout.write("{} {} {} \n".format(w1,w2,line[2])) 
            except:
                continue 
'''

In [ ]:
# debug
model = dconf_model
with open(sim999) as fin:
    lines = fin.readlines()
    for line in lines:
        line= line.split()
        print(line,avg_sim(line[0],line[1],model))
        continue
        try:
            if line[0] in model.vocab and line[1] in model.vocab:
                continue
        except:
            print(line,avg_sim(line[0],line[1],model))

In [17]:
datasets = [ws353A,rw,sim999,turk,mturk,rg,men]

In [6]:
vocab = set()
for ds in datasets:
    with open(ds) as fin:
        lines = fin.readlines()
        for line in lines:
            line=line.split()
            if len(line)!=3:
                continue
            vocab.add(line[0])
            vocab.add(line[1])

In [7]:
from gensim.models import KeyedVectors
from gensim.scripts.glove2word2vec import glove2word2vec
def trimmer(file):
    out_file_ = file+'_trim'
    with open(out_file_,'w') as fout:
        with open(file) as fin:
            lines = fin.readlines()
            for line in tqdm(lines,position=0):
                _line=line.split()
                if not _line:
                    continue
                if _line[0].split('#')[0] in vocab:
                    fout.write(line.strip()+'\n')
    out_file = out_file_+'ed'
    _ = glove2word2vec(out_file_, out_file)
    print('written:'+out_file)

In [8]:
trimmer(wnet)

100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 2614119/2614119 [01:16<00:00, 34388.31it/s]


written:/home/manni/embs/en_wiki_wnet_epoch5_300_w10_explicit.txt_trim


In [9]:
trimmer(dconf)

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 207018/207018 [00:05<00:00, 34548.98it/s]


written:/home/manni/wn3.0_sense_vectors.txt_trim


In [ ]:
with open(sim999) as fin:
    lines = fin.readlines()
    for l in lines[:10]:
        print(l)

In [14]:
def avg_sim(w1,w2,model,verbose=False,maxsim=False):
    range_ = list()
    vocab = model.vocab
    a = list()
    for word in vocab:
        if word.startswith(w1+'#'):
            a.append(word)
    if w1 in vocab:
        a.append(w1)
    b = list()
    for word in vocab:
        if word.startswith(w2+'#'):
            b.append(word)
    if w2 in vocab:
        b.append(w2)
    div = len(a)*len(b)
    if div == 0:
        return 0, range_
    sims = list()
    for i in a:
        for j in b:
            sim = model.similarity(i,j)
            range_.append(sim)
            if verbose:
                print(i,j,sim)
            sims+=[sim]
    if maxsim:
        return max(sims), range_
    else:
        return sum(sims)/div, range_
    
def get_corrs(lines,model):
    sims = list()
    scores = list()
    for line in lines:
        line = line.split()
        if not line or len(line)>3:
            continue
        sim,_ = avg_sim(line[0],line[1],model)
        if sim:
            score = float(line[2])
            sims.append(sim)
            scores.append(score)
    assert len(sims)==len(scores) and len(sims)>0, sims
    coverage = round((len(sims)/len(lines))*100,2)
    pcorr, _ = pearsonr(sims, scores)
    scorr, _ = spearmanr(sims, scores)
    return '%.2f' % (pcorr*100),'%.2f' % (scorr*100), '%.2f' % (coverage)

In [18]:
for ds in datasets:
    with open(ds) as fin:
        name = ds.split('/')[-1].split('.')[0]
        lines = fin.readlines()
        print(r'\begin{subsection}{'+name+r'}')
        print(r'\begin{table}[H]')
        print(r'\begin{tabular}{|l|c|c|c|}')
        print(r'\hline')
        print("Model & Pearsons & Spearmans & Coverage"+r"\\")
        print(r'\hline')
        for i,model in enumerate(models):
            result = get_corrs(lines,model)
            print(model_names[i],'& ',result[0],'& ',result[1],'& ',result[2],r'\\')
        print(r'\hline')
        print('\end{tabular}')
        print('\end{table}')
        print(r'\end{subsection}')
        print()

\begin{subsection}{EN-WS353}
\begin{table}[H]
\begin{tabular}{|l|c|c|c|}
\hline
Model & Pearsons & Spearmans & Coverage\\
\hline
wnet &  58.07 &  59.29 &  92.92 \\
dconf &  70.31 &  74.63 &  98.02 \\
\hline
\end{tabular}
\end{table}
\end{subsection}

\begin{subsection}{rw}
\begin{table}[H]
\begin{tabular}{|l|c|c|c|}
\hline
Model & Pearsons & Spearmans & Coverage\\
\hline
wnet &  14.82 &  17.83 &  80.38 \\
dconf &  52.48 &  52.58 &  67.65 \\
\hline
\end{tabular}
\end{table}
\end{subsection}

\begin{subsection}{EN-SIM999}
\begin{table}[H]
\begin{tabular}{|l|c|c|c|}
\hline
Model & Pearsons & Spearmans & Coverage\\
\hline
wnet &  31.74 &  32.28 &  99.40 \\
dconf &  52.60 &  50.62 &  100.00 \\
\hline
\end{tabular}
\end{table}
\end{subsection}

\begin{subsection}{EN_TRUK}
\begin{table}[H]
\begin{tabular}{|l|c|c|c|}
\hline
Model & Pearsons & Spearmans & Coverage\\
\hline
wnet &  59.69 &  60.21 &  81.53 \\
dconf &  74.99 &  75.55 &  58.89 \\
\hline
\end{tabular}
\end{table}
\end{subsection}

\

In [ ]:
import numpy as np
import scipy.stats
()

def mean_confidence_interval(data, confidence=0.95):
    a = 1.0 * np.array(data)
    n = len(a)
    m, se = np.mean(a), scipy.stats.sem(a)
    h = se * scipy.stats.t.ppf((1 + confidence) / 2., n-1)
    return m, h

In [ ]:
from multiprocessing import Pool
import workers
output = dict()
if __name__ == "__main__":
    for ds in tqdm(datasets,position=0):
        with open(ds) as fin:
            name = ds.split('/')[-1].split('.')[0]
            lines = fin.readlines()
            with Pool(processes=len(models)) as pool:
                results = list(pool.apply_async(workers.get_corrs, args=(lines,model)) for model in models)
                results = [r.get() for r in results]
                output[name]=results

In [ ]:
for name,v in output.items():
    print(r'\begin{subsection}{'+name+r'}')
    print(r'\begin{table}[H]')
    print(r'\begin{tabular}{|l|c|c|c|}')
    print('\hline')
    print("Model & Pearsons & Spearmans & Coverage"+r"\\")
    print('\hline')
    for i,scores in enumerate(v):
        print(model_names[i],end=' & ')
        print(scores[0],end=' & ')
        print(scores[1],end=' & ')
        print(str(scores[2]),end=r'\\')
        print()
    print('\hline')
    print('\end{tabular}')
    print('\end{table}')
    print(r'\end{subsection}')
    print()

In [ ]:
avg_sim('fast','rapid',dconf_model,verbose=True)

In [ ]:
avg_sim('fast','rapid',wnet_model,verbose=True)

In [ ]:
# Import libraries
import matplotlib.pyplot as plt
import numpy as np

In [ ]:
plt.clf()
# Creating axes instance
print('1. dconf, 2. SPX, 3.W2V')
with open(sim999) as fin:
    lines = fin.readlines()
    for line in lines[:5]:
        print(line)
        line = line.split()
        a = line[0]
        b = line[1]
        #plt.xticks(np.arange(3), ["DCONF", "SPX", "W2V"],rotation='vertical')
        sa,ra = avg_sim(a,b,dconf_model)
        sb,rb = avg_sim(a,b,spxPEM_model)
        sc,rc = avg_sim(a,b,w2v_model)
        data = [ra,rb,rc]
        bp = plt.boxplot(data)
        plt.show()

In [ ]:
a = 'old'
b = 'new'
avg_sim(a,b,dconf_model,verbose=True)

In [ ]:
avg_sim(a,b,spxPEM_model,verbose=True)

In [ ]:
phrases = ['new york','new zealand']
markers = ['#E','#P']
print(r'\begin{table}[!h]')
print(r'\begin{tabular}{|l|c|c|}')
print('\hline')
print("Model & Phrase & Similarity"+r"\\")
print('\hline')
for phrase in phrases:
    words = phrase.split()
    s1 = [word+'#E' for word in words]
    s2 = [word+'#P' for word in words]
    senses = [s1,s2]
    for i,model in enumerate(models):
        sim = model.similarity(words[0],words[1])
        print(model_names[i],end=' & ')
        print(phrase,end=' & ')
        print('%.2f' % (sim),end=r'\\')
        print()
        for sense in senses:
            if sense[0] in model.vocab and sense[1] in model.vocab:
                sim = model.similarity(sense[0],sense[1])
                print(model_names[i],end=' & ')
                print(' '.join(sense),end=' & ')
                print('%.2f' % (sim),end=r'\\')
                print()
    print('\hline')
print('\end{tabular}')
print('\end{table}')

In [ ]:
import scipy
print(scipy.__version__)

In [ ]:
from scipy.stats import bootstrap

In [ ]:
def viterbi_algorithm(observations, states, start_p, trans_p, emit_p):
    V = [{}]
    for st in states:
         V[0][st] = {"prob": start_p[st] * emit_p[st][observations[0]], "prev": None}

    for t in range(1, len(observations)):
        V.append({})
        for st in states:
            max_tr_prob = V[t - 1][states[0]]["prob"] * trans_p[states[0]][st]
            prev_st_selected = states[0]
            for prev_st in states[1:]:
                tr_prob = V[t - 1][prev_st]["prob"] * trans_p[prev_st][st]
                if tr_prob > max_tr_prob:
                    max_tr_prob = tr_prob
                    prev_st_selected = prev_st

            max_prob = max_tr_prob * emit_p[st][observations[t]]
            V[t][st] = {"prob": max_prob, "prev": prev_st_selected}
    for line in dptable(V):
        print(line)

    opt = []
    max_prob = 0.0
    best_st = None

    for st, data in V[-1].items():
        if data["prob"] > max_prob:
            max_prob = data["prob"]
            best_st = st
    opt.append(best_st)
    previous = best_st

    for t in range(len(V) - 2, -1, -1):
        opt.insert(0, V[t + 1][previous]["prev"])
        previous = V[t + 1][previous]["prev"]

    print ("The steps of states are " + " ".join(opt) + " with highest probability of %s" % max_prob)

def dptable(V):
     
    yield " ".join(("%12d" % i) for i in range(len(V)))
    for state in V[0]:
        yield "%.7s: " % state + " ".join("%.7s" % ("%f" % v[state]["prob"]) for v in V)

In [ ]:
observations = ("normal", "cold", "dizzy")
states = ("Healthy", "Fever")
start_p = {"Healthy": 1, "Fever": 1}
trans_p = {
    "Healthy": {"Healthy": 1, "Fever": 1},
    "Fever": {"Healthy": 1, "Fever": 1},
}
emit_p = {
    "Healthy": {"normal": 0.5, "cold": 0.4, "dizzy": 0.1},
    "Fever": {"normal": 0.1, "cold": 0.3, "dizzy": 0.6},
}

In [ ]:
viterbi_algorithm(observations, states, start_p, trans_p, emit_p)

In [ ]:
model = spxEM_model

In [ ]:
P = list()
w = list()
mat = {}

        

In [ ]:
import numpy as np

def viterbi(y, A, B, initial_probs = None):
    K = A.shape[0]
    initial_probs = initial_probs if initial_probs is not None else np.full(K, 1 / K)
    T = len(y)
    T1 = np.empty((K, T), 'd')
    T2 = np.empty((K, T), 'B')
    T1[:, 0] = initial_probs * B[:, y[0]]
    T2[:, 0] = 0
    
    for i in range(1, T):
        T1[:, i] = np.max(T1[:, i - 1] * A.T * B[np.newaxis, :, y[i]].T, 1)
        T2[:, i] = np.argmax(T1[:, i - 1] * A.T, 1)

    x = np.empty(T, 'B')
    x[-1] = np.argmax(T1[:, T - 1])
    
    for i in reversed(range(1, T)):
        x[i - 1] = T2[x[i], i]

    return x, T1, T2


In [ ]:
print(spxEM_model.similarity('new#E','york#E'))
print(spxEM_model.similarity('new#E','york'))
print(spxEM_model.similarity('new','york#E'))
print(spxEM_model.similarity('new','york'))

In [ ]:
print(spxPEM_model.similarity('new#E','york#E'))
print(spxPEM_model.similarity('new#E','york'))
print(spxPEM_model.similarity('new','york#E'))
print(spxPEM_model.similarity('new','york'))

In [ ]:
a = list()
b = list()
with open(sim999) as fin:
    name = ds.split('/')[-1].split('.')[0]
    lines = fin.readlines()
    for line in lines[:10]:
        line = line.split()
        score = avg_sim(line[0],line[1],dconf_model,maxsim=True)
        print(line,score)
        a.append(line[2])
        b.append(score)
spearmanr(a, b)

In [ ]:
a = list()
b = list()
with open(sim999) as fin:
    name = ds.split('/')[-1].split('.')[0]
    lines = fin.readlines()
    for line in lines[:10]:
        line = line.split()
        score = avg_sim(line[0],line[1],spxPEM_model,maxsim=True)
        print(line,score)
        a.append(line[2])
        b.append(score)
spearmanr(a, b)

# numsense

In [ ]:
def num_sim(w1,w2,model):
    vocab = model.vocab
    a = list()
    for word in vocab:
        if word.startswith(w1+'#'):
            a.append(word)
    if w1 in vocab:
        a.append(w1)
    b = list()
    for word in vocab:
        if word.startswith(w2+'#'):
            b.append(word)
    if w2 in vocab:
        b.append(w2)
    div = len(a)*len(b)
    if div == 0:
        return 0
    return 1/div

def get_ncorrs(lines,model):
    sims = list()
    scores = list()
    for line in lines:
        line = line.split()
        if not line or len(line)>3:
            continue
        sim = num_sim(line[0],line[1],model)
        if sim:
            score = float(line[2])
            sims.append(sim)
            scores.append(score)
    assert len(sims)==len(scores) and len(sims)>0
    coverage = round((len(sims)/len(lines))*100,2)
    pcorr, _ = pearsonr(sims, scores)
    scorr, _ = spearmanr(sims, scores)
    return '%.2f' % (pcorr*100),'%.2f' % (scorr*100), coverage

In [ ]:
output_ = dict()
for ds in tqdm(datasets,position=0):
    with open(ds) as fin:
        name = ds.split('/')[-1].split('.')[0]
        lines = fin.readlines()
        results = get_ncorrs(lines,dconf_model)
        output_[name]=results

In [ ]:
for name,v in output_.items():
    print(r'\begin{subsection}{'+name+r'}')
    print(r'\begin{table}[H]')
    print(r'\begin{tabular}{|l|c|c|}')
    print('\hline')
    print("Pearsons & Spearmans & Coverage"+r"\\")
    print('\hline')
    print(v[0],end=' & ')
    print(v[1],end=' & ')
    print(str(v[2]),end=r'\\')
    print()
    print('\hline')
    print('\end{tabular}')
    print('\end{table}')
    print(r'\end{subsection}')
    print()

# HMM

In [ ]:
model = spxPEM_model
#model = w2v_model

In [ ]:
#sentence = 'new york city is composed of five boroughs'
#sentence = 'the south american jaguar is a jaguar population in south america'
sentence = 'the central bank of brazil is the central bank and the issuing authority'
sentence = sentence.split()
sentence_E = [word+'#E' for word in sentence]
sentence_P = [word+'#P' for word in sentence]
sentences = [sentence,sentence_E,sentence_P]
#sentences = [sentence,sentence_E]

In [ ]:
best_tuples = list()
z = 1
path = list()
for i in range(len(sentence)-1):
    scores = list()
    candidates = list()
    for j in range(len(sentences)):
        for k in range(len(sentences)):
            edge = model.similarity(sentences[j][i],sentences[k][i+1])
            scores.append(edge)
            candidates.append([sentences[j][i],sentences[k][i+1]])
    z = max(scores)
    _best = tuple(candidates[np.argmax(scores)])
    print(scores,_best)
    print()
    best_tuples.append(_best)

In [ ]:
mat.shape

In [ ]:
import itertools
model = spxPEM_model
sequences = [seq for seq in itertools.product([0,1], repeat=len(sentence))]
best = None
m_score = 0
for seq in sequences:
    sent = list()
    score = 0
    for i,j in zip(seq,range(len(sentence))):
        sent.append(sentences[i][j])
    for i in range(len(sent)-1): 
        score+=model.similarity(sent[i],sent[i+1])
    if score>m_score:
        m_score=score
        best=sent
print(best,score)

In [ ]:
#Binary sequence

import itertools
sequences = [seq for seq in itertools.product([0,1], repeat=len(sentence))]
best = None
m_score = 0
for seq in sequences:
    sent = list()
    score = 0
    for i,j in zip(seq,range(len(sentence))):
        sent.append(sentences[i][j])
    for i in range(len(sent)-1):
        if (sent[i],sent[i+1]) in best_tuples:
            score+=1
    for i in range(len(sent)-1,1,-1):
        if (sent[i-1],sent[i]) in best_tuples:
            score+=1
    if score>m_score:
        m_score=score
        best=sent
        print(best,score)

In [ ]:
best

In [ ]:
truth = ['new#E', 'york#E', 'city#E', 'is', 'composed', 'of', 'five', 'boroughs#E']
score = 0
for i in range(len(truth)-1): 
    score+=model.similarity(truth[i],truth[i+1])
print(score)

In [ ]:
import pickle

a = dict()

for sent in sentences:
    for word in sent:
        if word in model.vocab:
            a[word]=model.get_vector(word)

with open('spx.pkl', 'wb') as handle:
    pickle.dump(a, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [ ]:
with open('spx.pkl', 'rb') as handle:
    b = pickle.load(handle)
    print(b)

In [ ]:
data = [['bookcase','fridge','wood',1],['bucket','mug','round',0],['angle','curve','sharp',1],\
        ['pelican','turtle','water',0],['wire','coil','metal',0],['apple','banana','red',1]]

In [ ]:
model = spxEM_model

for items in data:
    s1 = model.similarity(items[0],items[2])
    s2 = model.similarity(items[1],items[2])
    print('NE:',s1,s2,items[3])
    if items[0]+'#E' in model.vocab and items[2]+'#E' in model.vocab:
        s1 = model.similarity(items[0]+'#E',items[2]+'#E')
    else:
        s1 = 0
    if items[1]+'#E' in model.vocab and items[2]+'#E' in model.vocab:
        s2 = model.similarity(items[1]+'#E',items[2]+'#E')
    else:
        s2 = 0
    print('E:',s1,s2,items[3])
    print()

In [ ]:
model = w2v_model

for items in data:
    s1 = model.similarity(items[0],items[2])
    s2 = model.similarity(items[1],items[2])
    print('NE:',s1,s2,items[3])
    if items[0]+'#E' in model.vocab and items[2]+'#E' in model.vocab:
        s1 = model.similarity(items[0]+'#E',items[2]+'#E')
    else:
        s1 = 0
    if items[1]+'#E' in model.vocab and items[2]+'#E' in model.vocab:
        s2 = model.similarity(items[1]+'#E',items[2]+'#E')
    else:
        s2 = 0
    print(s1,s2,items[3])
    print()

# entities

In [ ]:
e_file = '/home/manni/data/entities/entities.tsv'

In [ ]:
entities = set()
bi_ent = set()
with open(e_file,'r') as fin:
    lines = fin.readlines()
    for line in lines:
        if line.startswith('#'):
            continue
        ent = line.split('\t')[0].lower()
        entities.add(ent)
        if len(ent.split())==2:
            bi_ent.add(ent)

In [ ]:
print(len(bi_ent))
print(len(entities))

In [ ]:
common_vocab = set()
for model in models:
    if not common_vocab:
        common_vocab.update(set(model.vocab.keys()))
    else:
        common_vocab = common_vocab & set(model.vocab.keys())
print(len(common_vocab))

In [ ]:
import random
scores = dict()
found = len(bi_ent)
for i,_ in enumerate(models):
    scores[i]=0
for ent in tqdm(bi_ent):
    words = ent.split()
    if len(set(words) & common_vocab) != 2:
        found-=1
        continue
    rands = random.choices(list(common_vocab), k=3)
    for i,model in enumerate(models):
        high=True
        sim = model.similarity(words[0],words[1])
        for rand in rands:
            _sim = model.similarity(words[0],rand)
            if _sim>sim:
                high=False
                break
        if high:
            scores[i]+=1

In [ ]:
print(scores)
print(found)

In [ ]:
def get_score(model,anotate=''):
    vocab = set(model.vocab.keys())
    rands = random.choices(list(vocab), k=5)
    score = 0
    found = len(bi_ent)
    for ent in tqdm(bi_ent,position=0):
        words = ent.split()
        words = [word+anotate for word in words]
        if len(set(words) & vocab) != 2:
            found-=1
            continue
        #rands = random.choices(list(vocab), k=3)
        #rands = [random.choice(list(vocab))]
        high=True
        sim = model.similarity(words[0],words[1])
        for rand in rands:
            _sim = model.similarity(words[0],rand)
            if _sim>sim:
                high=False
                break
        if high:
            score+=1
    return round((score/found)*100,2)

In [ ]:
w_score = get_score(models[0])
print(w_score)

In [ ]:
e_score = get_score(models[1],anotate='#E')
print(e_score)

In [ ]:
p_score = get_score(models[2],anotate='#E')

In [ ]:
print(p_score)

In [ ]:
p_score = get_score(models[2],anotate='#P')

In [ ]:
print(p_score)

In [ ]:
e_score = get_score(models[1])
print(e_score)

# analogies

In [ ]:
file = '/home/manni/data/wordsim/google_anal.txt'

In [ ]:
import numpy as np
def cosmul(a,b,c,model):
    vocab = set(model.vocab.keys())
    vocab = list(vocab-{a,b,c})
    scores = list()
    for v in vocab:
        score = ((model.similarity(v,b)+1)*(model.similarity(v,c)+1))/(2*(model.similarity(v,a)+1)+0.004)
        scores.append(score)
    maxi = np.argmax(scores)
    return vocab[maxi]

In [ ]:
def get_score(model):
    total = 0
    score = 0
    with open(file) as fin:
        lines = fin.readlines()[:100]
        for line in tqdm(lines,position=0):
            line = line.lower().split()
            if len(line)<4:
                continue
            res = cosmul(line[0],line[1],line[2],model)
            if res == line[3]:
                score+=1
            total+=1
    print(round((score/total)*100,2))

In [ ]:
get_score(spxPEM_model)

In [ ]:
model = spxPEM_model

In [ ]:
from multiprocessing import Pool
import workers
if __name__ == "__main__":
    with open(file) as fin:
        lines = fin.readlines()
        lines_ = list()
        for line in tqdm(lines,position=0):
            line = line.lower().split()
            if len(line)<4:
                continue
            lines_.append(line)
        with Pool(processes=20) as pool:
            results = list(pool.apply_async(workers.cosmul, args=(line,model)) for line in lines_)
            results = [r.get() for r in tqdm(results,position=0)]

In [ ]:
sum(results)/len(results)

# sandbox

In [ ]:
spxPEM_model.similarity('new#E','york#E')